In [29]:
import pennylane as qml
import matplotlib.pyplot as plt
import numpy as np

In [30]:
#Set seed:
RANDOM_STATE = 42
seed = np.random.seed(RANDOM_STATE)

num_qubits = 2
x = 0
label = 0
l = 2
theta = 2*np.pi*np.random.random(num_qubits*l*3)

In [31]:
#Create rotation angle functions
def phi(x,qubit):
    return 2*qubit*np.arccos(x)

def phi_plus(x,qubit): 
    return 2*qubit*np.arccos(x) + np.pi/2

def phi_min(x,qubit):
    return 2*qubit*np.arccos(x) - np.pi/2

#Create feature map
def U(x,num_qubits, label):
    assert label <= 2*num_qubits, "Label cannot be bigger than 2*num_qubits!"

    #Return U if label = 0
    if label == 0:
        for i in range(num_qubits):
            qml.RY(phi(x,i),i, id = round(phi(x,i),2))

    #Return U+ (for C+) if label in [1,num_qubits]
    elif label >= 1 and label <= num_qubits:
        for i in range(num_qubits):
            if i == label-1:
                qml.RY(phi_plus(x,i),i, id = round(phi_plus(x,i),2))
            else:
                qml.RY(phi(x,i),i, id = round(phi(x,i),2))

    #Return U- (for C-) if label in [num_qubits+1,2*num_qubits]
    else:
        for i in range(num_qubits):
            if i == label-num_qubits-1:
                qml.RY(phi_min(x,i),i, id = round(phi_min(x,i),2))
            else:
                qml.RY(phi(x,i),i, id = round(phi(x,i),2))
    
    return 0

#Define the Hamiltonian
def H(n_qubits):
    ops = qml.operation.Tensor(*[qml.PauliZ(i) for i in range(n_qubits)])
    return ops

#Define circuit for the variational quantum classifier
def theta_gate(theta, i):
    
    assert len(theta)==3, "theta incorrect size"
    
    qml.RZ(theta[0], i, id = round(theta[0],2))
    qml.RX(theta[1], i, id = round(theta[1],2))
    qml.RZ(theta[2], i, id = round(theta[2],2))
    return 0

def entangling_gate(n):
    for i in range(0, n-1, 2):
            qml.CNOT([i, i+1])
    for i in range(1, n-1, 2):
            qml.CNOT([i, i+1])
    return 0

def HEA(theta, n, l):
    Theta = np.reshape(theta, (n,l,3))
    
    for i in range(l):
        for j in range(n):
            theta_gate(Theta[j,i,:], j) 
        entangling_gate(n)
        qml.Barrier()
    return 0

def build_circuit(x,num_qubits, label, theta, l):
    U(x, num_qubits, label)
    qml.Barrier()
    HEA(theta, num_qubits, l)
    hamiltonian = H(num_qubits)
    return qml.expval(hamiltonian)

In [32]:
def f_func(x,num_qubits,theta, l):
    label = 0
    dev = qml.device('default.qubit', wires=list(range(num_qubits)))
    circuit = qml.QNode(build_circuit, dev)
    result = circuit(x, num_qubits, label, theta, l)
    # qml.drawer.use_style('black_white')
    # fig, ax = qml.draw_mpl(circuit)(x, num_qubits, label, theta, l)
    # plt.show()
    return result

f_func(x,num_qubits,theta,l)

def dphi(x):
    return -1/np.sqrt(1-x**2)

def df_func(x,num_qubits,theta,l):
    C_plus = 0 
    C_minus = 0
    for i in range(1,2*num_qubits+1):
        if i <= num_qubits:
            dev = qml.device('default.qubit', wires=list(range(num_qubits)))
            circuit = qml.QNode(build_circuit, dev)
            result = circuit(x, num_qubits, i, theta, l)
            # qml.drawer.use_style('black_white')
            # fig, ax = qml.draw_mpl(circuit)(x, num_qubits, i, theta, l)
            # plt.show()
            C_plus += result
        else:
            dev = qml.device('default.qubit', wires=list(range(num_qubits)))
            circuit = qml.QNode(build_circuit, dev)
            result = circuit(x, num_qubits, i, theta, l)
            # qml.drawer.use_style('black_white')
            # fig, ax = qml.draw_mpl(circuit)(x, num_qubits, i, theta, l)
            # plt.show()
            C_minus += result

    return 1/4*dphi(x)*(C_plus-C_minus)

df_func(x,num_qubits,theta,l)

def func_and_deriv(x, num_qubits, theta, l):
    f = []
    df = []

    for i in x:
        f.append(f_func(i,num_qubits,theta,l))
        df.append(df_func(i,num_qubits,theta,l))

    return np.array(f), np.array(df)

f, df = func_and_deriv(np.linspace(0,0.9,20),num_qubits,theta,l)



[-0.59393023 -0.60558236 -0.61180717 -0.6125068  -0.60758115 -0.5969266
 -0.58043455 -0.55798972 -0.52946802 -0.49473381 -0.45363638 -0.406005
 -0.35164211 -0.29031324 -0.22173152 -0.14553263 -0.06123143  0.03186046
  0.13481147  0.249528  ]
[-0.14927877 -0.12308578 -0.09588132 -0.06764808 -0.03835626 -0.00796209
  0.02359457  0.05639361  0.09053978  0.12616997  0.16346414  0.20266185
  0.24408818  0.28819622  0.33563997  0.38740838  0.44509258  0.51148165
  0.59211597  0.7004428 ]
